In [1]:
import cohere
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
CO_API_KEY = os.environ.get("CO_API_KEY")
co = cohere.ClientV2(CO_API_KEY)

In [9]:
PROMPT_RUBRIK = '''

Given a prompt, can you please improve it so that it follows the Prompt Rubric:

1. First, break the prompt into sequential steps: Eg. First [do something], then [do something]. Finally, [do something].

2. Then, provide necessary context or background information. Eg. Report card. Which report card? For elementary students in Brazil?

3. Afterwards, assign a role to the AI model. Eg. Imagine you are a teacher and I am a student. Please explain Newton’s Law.

4. After this, specify the tone (Please use constructive tone, related to roles: whose perspective
should the response be written from and who is the audience.). If possible, use action words to be more specific.

5. If there are no examples, add an example when applicable.

6. Explicitly say format of the response expected (bullet points, tables, code, etc).

7. Include this at the end of the prompt: “Please do not assume anything, ask me any
clarification questions if need be.

8. If there is code included, label the prompt at the top and include ALL the code below the prompt.

9. Finally, please do not apply it to this prompt. Memorize the Prompt Rubric for future prompts.
'''
PROMPT_ADDON = '''
\n Improve this prompt using the Prompt Rubric and Context Rubric, with the response being a string of the new prompt only. 

If there is code included, put the prompt at the top and include ALL the code below the prompt.

Do not include **Improved Prompt**
'''

In [29]:
def engineer_context(prompt_type: str) -> str:
    if prompt_type == 'EDUCATIONAL':
        return '''You are to memorize the following Context Rubric for future prompts only.
        Context Rubric:
        1. If the prompt asks for a direct answer, implement guide rails in the prompt to prevent a direct answer.
        2. Ensure the prompt will activate chain-of-thought prompting when obtaining a response.'''
    elif prompt_type == 'PROBLEM-SOLVING':
        return '''You are to memorize the following Context Rubric for future prompts only.
        Context Rubric:
        1. If the prompt asks for a direct answer or multiple choice, use zero-shot prompting in the prompt
        2. If the prompt asks to solve a complicated problem, use meta prompting in the prompt'''
    else: # prompt_type == EVERYDAY
        return '''You are to memorize the following Context Rubric for future prompts only.
        Context Rubric:
        1. Ensure the prompt will activate few-shot prompting when obtaining a response
        2. Make the prompt elicit a concise response without overcomplicating it.'''

In [34]:
PROMPT_TYPE = engineer_context('PROBLEM-SOLVING')
user_prompt = '''prove CLT'''
USER_PROMPT_FINAL = user_prompt + PROMPT_ADDON

In [35]:
pipeline = [PROMPT_RUBRIK, PROMPT_TYPE, USER_PROMPT_FINAL]

for step in pipeline:
    response = co.chat(model="command-a-03-2025", messages=[{"role": "user", "content": step}])
    print('begin next\n')
    print(response.message.content[0].text)

begin next

Understood! I’ll memorize the **Prompt Rubric** for future prompts and ensure they are structured as follows:

1. **Break into sequential steps**: Clearly outline the steps to be followed (e.g., First [do something], then [do something], finally [do something]).  
2. **Provide necessary context**: Include relevant background information (e.g., specific details about the subject, audience, or scenario).  
3. **Assign a role to the AI model**: Define the AI’s role (e.g., teacher, consultant, writer) and the user’s role (e.g., student, client, reader).  
4. **Specify the tone**: Indicate the tone and perspective (e.g., constructive, formal, conversational) and the intended audience.  
5. **Add examples if applicable**: Include examples to clarify expectations.  
6. **Specify the response format**: Request the output format (e.g., bullet points, paragraphs, tables, code).  
7. **Include clarification request**: End with “Please do not assume anything, ask me any clarification q

In [55]:
#prompt = '''explain how to implement Djikstra’s algorithm in C'''
prompt = '''explain object-oriented programming'''

In [56]:
prompt = prompt + 'Improve this prompt using the Prompt Rubric, with the response being a string of the new prompt only.'

In [57]:
response = co.chat(model="command-a-03-2025", messages=[{"role": "user", "content": prompt}])
print(response.message.content[0].text)

"Explain the core principles of object-oriented programming (OOP), including encapsulation, inheritance, polymorphism, and abstraction, and provide examples of how these concepts are applied in real-world programming scenarios."
